In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc
from sklearn.datasets import load_breast_cancer

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel

np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc

In [2]:
data = load_breast_cancer()
dfx = pd.DataFrame(data.data, columns= data.feature_names)
y = pd.Series(data.target).map({0:'malignant', 1:'benign'})
dfy = pd.get_dummies(y) 
X = dfx.values.astype(float)
Y = dfy.values.astype(float)

In [3]:
mp = AggMap(dfx, metric = 'correlation')
mp.fit(cluster_channels = 5, verbose = 0)

2020-10-25 13:21:53,302 - INFO - [bidd-aggmap] - Calculating distance ...
2020-10-25 13:21:53,303 - INFO - [bidd-aggmap] - the number of process is 16


100%|##########| 30/30 [00:00<00:00, 697.30it/s]


2020-10-25 13:21:53,758 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...
2020-10-25 13:21:56,955 - INFO - [bidd-aggmap] - Applying grid feature map(assignment), this may take several minutes(1~30 min)
2020-10-25 13:21:56,965 - INFO - [bidd-aggmap] - Finished


## 5-fold cross validation and repeat 5 times 

In [4]:
outer_fold = 5

run_all = []
for repeat_seed in [8, 16, 32, 64, 128]: #5 repeats random seeds
    
    outer = KFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(range(len(Y)), Y[:,0])

    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testx = X[test_idx]
        
        trainx = X[train_idx]
        trainY = Y[train_idx]

        trainX = mp.batch_transform(trainx, scale_method = 'standard')
        testX = mp.batch_transform(testx, scale_method = 'standard')
        
        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        clf = AggModel.MultiClassEstimator(epochs = 50, gpuid = 3, 
                                           batch_size = 8, verbose = 0) #conv1_kernel_size = 5,
        clf.fit(trainX, trainY)

        y_true = testY[:,0]
        y_pred = clf.predict(testX)[:,0]
        y_score = clf.predict_proba(testX)[:,0]

        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        
        acc = (tp + tn) / sum([tn, fp, fn, tp])
        
        sensitivity = tp / sum([tp, fn])
        specificity = tn / sum([tn, fp])
        
        prc_auc = prc_auc_score(y_true, y_score)
        roc_auc = roc_auc_score(y_true, y_score)
        
        precision = tp / sum([tp, fp])
        recall =  tp / sum([tp, fn]) #equals to sensitivity
        
        
        res = {'fold': fold_num,
               'repeat_seed':repeat_seed,
               
               'accuracy':acc, 
               
               'prc_auc':prc_auc, 
               'roc_auc':roc_auc,

               'sensitivity': sensitivity, 
               'specificity': specificity,
               
               'precision':precision,
               'recall':recall,
               'F1': 2*precision*sensitivity/(precision+sensitivity)
              }
        
        run_all.append(res)

100%|##########| 114/114 [00:00<00:00, 1502.05it/s]



 input train and test X shape is (455, 6, 5, 5), (114, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=50, gpuid='3')


100%|##########| 114/114 [00:00<00:00, 991.65it/s]



 input train and test X shape is (455, 6, 5, 5), (114, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=50, gpuid='3')


100%|##########| 114/114 [00:00<00:00, 1519.00it/s]


...


 input train and test X shape is (455, 6, 5, 5), (114, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=50, gpuid='3')


100%|##########| 114/114 [00:00<00:00, 1137.82it/s]



 input train and test X shape is (455, 6, 5, 5), (114, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=50, gpuid='3')


100%|##########| 113/113 [00:00<00:00, 1447.45it/s]



 input train and test X shape is (456, 6, 5, 5), (113, 6, 5, 5) 
MultiClassEstimator(batch_size=8, epochs=50, gpuid='3')


In [5]:
clf._model.count_params()

323138

In [6]:
df = pd.DataFrame(run_all)
df.to_excel('results.xlsx')


In [7]:
df.groupby('repeat_seed').apply(np.mean).mean().round(3)

repeat_seed    49.600
accuracy        0.972
prc_auc         0.996
roc_auc         0.994
sensitivity     0.980
specificity     0.955
precision       0.975
recall          0.980
F1              0.978
dtype: float64

In [8]:
df.groupby('repeat_seed').apply(np.std).mean().round(3)

repeat_seed    0.000
accuracy       0.014
prc_auc        0.005
roc_auc        0.007
sensitivity    0.013
specificity    0.032
precision      0.015
recall         0.013
F1             0.011
dtype: float64